# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
#from scipy.misc import imread, imresize
from imageio import imread
from skimage.transform import resize
import datetime
import os

import warnings
warnings.filterwarnings('ignore')

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from tensorflow import keras
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('/home/datasets/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/home/datasets/Project_data/val.csv').readlines())
batch_size = 40 #experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [4]:
# Let we do the generators and input the images as we see that our images have two different sizes. 
x = 30 # No. of frames images
y = 120 # Width of the image
z = 120 # height

def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [i for i in range(0,x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    # Let us resize all the images 
                    Temp_img = resize(image,(y,z))
                    Temp_img = Temp_img/127.5-1 
                    
                    batch_data[folder,idx,:,:,0] = (Temp_img[:,:,0])#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (Temp_img[:,:,1])#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (Temp_img[:,:,2])#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches.
        
# Let us see that if the folder is not equal to the batch size * num of batches
        
        if (len(folder_list) != batch_size*num_batches):
            print("Batches: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    Temp_img = resize(image,(y,z))
                    Temp_img = Temp_img/127.5-1 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (Temp_img[:,:,0])
                    batch_data[folder,idx,:,:,1] = (Temp_img[:,:,1])
                    batch_data[folder,idx,:,:,2] = (Temp_img[:,:,2])
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [5]:
curr_dt_time = datetime.datetime.now()
train_path = '/home/datasets/Project_data/train'
val_path = '/home/datasets/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 30#num_epochs # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 30


In [6]:
# Let us import all the needed libraries of Keras.
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation,Dropout
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from keras.layers.convolutional import Conv3D, MaxPooling3D, Conv2D, MaxPooling2D
from keras.layers.recurrent import LSTM

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [7]:
# Let us see the Validate the Losses and put back the checkpoint
model_name = 'model_init_Conv_3D' + '' + str(curr_dt_time).replace(' ','').replace(':','') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model_epoch-{epoch:05d}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto',save_freq = num_epochs)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, mode='auto', min_lr=0.00001)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [8]:
# Let us see that the steps_per_epoch and validation steps are used by fit_generator to decide the no. of next()

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [9]:
# Let us train and validate the model 

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

## Exp-1 Image size 120X120, Batch size = 32, epochs = 30, [16,32,64,128,256]

In [10]:
# Let us experiment different x,y,z value in the CNNLSTM network and find tune all the image size & Hyperparameters later

x = 30 # number of frames
y = 120 # image width
z = 120 # image height

# Input all the images sequencial by building the layer with dropouts and batchnormalisation

model_ex13 = Sequential()   
model_ex13.add(TimeDistributed(Conv2D(16, (3, 3),padding='same', activation='relu'),input_shape=(x,y,z,3)))
model_ex13.add(TimeDistributed(BatchNormalization()))
model_ex13.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model_ex13.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
model_ex13.add(TimeDistributed(BatchNormalization()))
model_ex13.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model_ex13.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model_ex13.add(TimeDistributed(BatchNormalization()))
model_ex13.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model_ex13.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
model_ex13.add(TimeDistributed(BatchNormalization()))
model_ex13.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model_ex13.add(TimeDistributed(Conv2D(256, (3, 3) , padding='same', activation='relu')))
model_ex13.add(TimeDistributed(BatchNormalization()))
model_ex13.add(TimeDistributed(MaxPooling2D((2, 2))))

# Flatten layer 

model_ex13.add(TimeDistributed(Flatten()))

model_ex13.add(LSTM(64))
model_ex13.add(Dropout(0.25))

# Dense layer 
model_ex13.add(Dense(64,activation='relu'))
model_ex13.add(Dropout(0.25))
# Softmax layer

model_ex13.add(Dense(5, activation='softmax'))

# Adam optimiser

optimiser = Adam(lr=0.001) #write your optimizer
model_ex13.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_ex13.summary())

2024-08-02 05:40:07.243882: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2024-08-02 05:40:07.243967: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14800 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:1c:00.0, compute capability: 7.5


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 30, 120, 120, 16)  448      
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 30, 120, 120, 16)  64       
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (None, 30, 60, 60, 16)   0         
 tributed)                                                       
                                                                 
 time_distributed_3 (TimeDis  (None, 30, 60, 60, 32)   4640      
 tributed)                                                       
                                                                 
 time_distributed_4 (TimeDis  (None, 30, 60, 60, 32)   1

In [12]:
# Let us train and validate the model 
batch_size = 32
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [13]:

model_ex13.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=30, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 32
Epoch 1/30


2024-08-02 05:40:31.250460: I tensorflow/stream_executor/cuda/cuda_dnn.cc:377] Loaded cuDNN version 8302


17/17 [==============================] - ETA: 0s - loss: 1.5319 - categorical_accuracy: 0.3382Source path =  /home/datasets/Project_data/val ; batch size = 32
Batches:  4 Index: 32
17/17 [==============================] - 178s 11s/step - loss: 1.5319 - categorical_accuracy: 0.3382 - val_loss: 1.6144 - val_categorical_accuracy: 0.1875 - lr: 0.0010
Epoch 2/30
12/17 [====================>.........] - ETA: 33s - loss: 1.3207 - categorical_accuracy: 0.4488
Epoch 00002: saving model to model_init_Conv_3D2024-08-02053957.092705/model_epoch-00002.h5
17/17 [==============================] - 112s 7s/step - loss: 1.3136 - categorical_accuracy: 0.4306 - val_loss: 1.8708 - val_categorical_accuracy: 0.1667 - lr: 0.0010
Epoch 3/30
17/17 [==============================] - 110s 7s/step - loss: 1.1928 - categorical_accuracy: 0.4987 - val_loss: 1.6156 - val_categorical_accuracy: 0.2500 - lr: 0.0010
Epoch 4/30
 8/17 [=============>................] - ETA: 42s - loss: 1.1033 - categorical_accuracy: 0.5789


## Exp 2 Image size 120X120 , Batch size = 32, epochs = 30, [8,16,32,64,128]

In [7]:
# Let us experiment different x,y,z value in the CNNLSTM network and find tune all the image size & Hyperparameters later

x = 30 # number of frames
y = 120 # image width
z = 120 # image height

# Input all the images sequencial by building the layer with dropouts and batchnormalisation

model_ex18 = Sequential()   
model_ex18.add(TimeDistributed(Conv2D(8, (3, 3),padding='same', activation='relu'),input_shape=(x,y,z,3)))
model_ex18.add(TimeDistributed(BatchNormalization()))
model_ex18.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model_ex18.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu')))
model_ex18.add(TimeDistributed(BatchNormalization()))
model_ex18.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model_ex18.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
model_ex18.add(TimeDistributed(BatchNormalization()))
model_ex18.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model_ex18.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model_ex18.add(TimeDistributed(BatchNormalization()))
model_ex18.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model_ex18.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
model_ex18.add(TimeDistributed(BatchNormalization()))
model_ex18.add(TimeDistributed(MaxPooling2D((2, 2))))

# Flatten layer 

model_ex18.add(TimeDistributed(Flatten()))

model_ex18.add(LSTM(64))
model_ex18.add(Dropout(0.25))

# Dense layer 
model_ex18.add(Dense(64,activation='relu'))
model_ex18.add(Dropout(0.25))
# Softmax layer

model_ex18.add(Dense(5, activation='softmax'))

# Adam optimiser

optimiser = Adam(lr=0.001) #write your optimizer
model_ex18.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_ex18.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_16 (TimeDi  (None, 30, 120, 120, 8)  224       
 stributed)                                                      
                                                                 
 time_distributed_17 (TimeDi  (None, 30, 120, 120, 8)  32        
 stributed)                                                      
                                                                 
 time_distributed_18 (TimeDi  (None, 30, 60, 60, 8)    0         
 stributed)                                                      
                                                                 
 time_distributed_19 (TimeDi  (None, 30, 60, 60, 16)   1168      
 stributed)                                                      
                                                                 
 time_distributed_20 (TimeDi  (None, 30, 60, 60, 16)  

In [42]:
# Let us train and validate the model 
batch_size = 32
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [ ]:
# Let us fit the model

model_ex18.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=30, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50
14/21 [===================>..........] - ETA: 41s - loss: 1.0618 - categorical_accuracy: 0.5546
Epoch 00001: saving model to model_init_Conv_3D2024-08-01144433.743415/model_epoch-00001.h5
21/21 [==============================] - 127s 6s/step - loss: 1.0744 - categorical_accuracy: 0.5406 - val_loss: 1.0473 - val_categorical_accuracy: 0.6250 - lr: 1.0000e-05
Epoch 2/50
21/21 [==============================] - 116s 6s/step - loss: 1.0375 - categorical_accuracy: 0.5798 - val_loss: 0.9948 - val_categorical_accuracy: 0.5625 - lr: 1.0000e-05
Epoch 3/50
 2/21 [=>............................] - ETA: 1:25 - loss: 0.9419 - categorical_accuracy: 0.6471
Epoch 00003: saving model to model_init_Conv_3D2024-08-01144433.743415/model_epoch-00003.h5
21/21 [==============================] - 122s 6s/step - loss: 0.9935 - categorical_accuracy: 0.6078 - val_loss: 1.4573 - val_categorical_accuracy: 0.3750 - lr: 1.0000e-05
Epoch 4/50
11/21 [==============>...............] - ETA: 51s - loss: 1.0350 -

## Exp - 2 Image size 120X120,16 frames, Batch size = 32, epochs = 30, [8,16,32,64,128], Reduce the frame size

In [21]:
# Let us experiment different x,y,z value in the CNNLSTM network and find tune all the image size & Hyperparameters later

x = 16 # number of frames
y = 120 # image width
z = 120 # image height

# Input all the images sequencial by building the layer with dropouts and batchnormalisation

model_ex20 = Sequential()   
model_ex20.add(TimeDistributed(Conv2D(8, (3, 3),padding='same', activation='relu'),input_shape=(x,y,z,3)))
model_ex20.add(TimeDistributed(BatchNormalization()))
model_ex20.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model_ex20.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu')))
model_ex20.add(TimeDistributed(BatchNormalization()))
model_ex20.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model_ex20.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
model_ex20.add(TimeDistributed(BatchNormalization()))
model_ex20.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model_ex20.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model_ex20.add(TimeDistributed(BatchNormalization()))
model_ex20.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model_ex20.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
model_ex20.add(TimeDistributed(BatchNormalization()))
model_ex20.add(TimeDistributed(MaxPooling2D((2, 2))))

# Flatten layer 

model_ex20.add(TimeDistributed(Flatten()))

model_ex20.add(LSTM(64))
model_ex20.add(Dropout(0.25))

# Dense layer 
model_ex20.add(Dense(64,activation='relu'))
model_ex20.add(Dropout(0.25))
# Softmax layer

model_ex20.add(Dense(5, activation='softmax'))

# Adam optimiser

optimiser = Adam(lr=0.001) #write your optimizer
model_ex20.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_ex20.summary())
        


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_48 (TimeDi  (None, 16, 120, 120, 8)  224       
 stributed)                                                      
                                                                 
 time_distributed_49 (TimeDi  (None, 16, 120, 120, 8)  32        
 stributed)                                                      
                                                                 
 time_distributed_50 (TimeDi  (None, 16, 60, 60, 8)    0         
 stributed)                                                      
                                                                 
 time_distributed_51 (TimeDi  (None, 16, 60, 60, 16)   1168      
 stributed)                                                      
                                                                 
 time_distributed_52 (TimeDi  (None, 16, 60, 60, 16)  

In [23]:
batch_size = 32
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [19]:
model_ex20.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=30, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 32
Epoch 1/30
17/17 [==============================] - ETA: 0s - loss: 1.6000 - categorical_accuracy: 0.2629Source path =  /home/datasets/Project_data/val ; batch size = 32
Batches:  4 Index: 32
17/17 [==============================] - 134s 8s/step - loss: 1.6000 - categorical_accuracy: 0.2629 - val_loss: 1.5588 - val_categorical_accuracy: 0.3229 - lr: 0.0010
Epoch 2/30
12/17 [====================>.........] - ETA: 20s - loss: 1.4334 - categorical_accuracy: 0.3630
Epoch 00002: saving model to model_init_Conv_3D2024-08-02053957.092705/model_epoch-00002.h5
17/17 [==============================] - 67s 4s/step - loss: 1.4173 - categorical_accuracy: 0.3780 - val_loss: 1.5717 - val_categorical_accuracy: 0.3333 - lr: 0.0010
Epoch 3/30
17/17 [==============================] - 68s 4s/step - loss: 1.2862 - categorical_accuracy: 0.4674 - val_loss: 1.6602 - val_categorical_accuracy: 0.2500 - lr: 0.0010
Epoch 4/30
 8/17 [=============>

## Exp - 3 Increase Epochs

In [20]:
model_ex20.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=40, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/40
17/17 [==============================] - 31s 2s/step - loss: 0.9362 - categorical_accuracy: 0.6713 - val_loss: 0.9041 - val_categorical_accuracy: 0.5000 - lr: 1.0000e-05
Epoch 2/40
12/17 [====================>.........] - ETA: 8s - loss: 0.9634 - categorical_accuracy: 0.6422 
Epoch 00002: saving model to model_init_Conv_3D2024-08-02053957.092705/model_epoch-00002.h5
17/17 [==============================] - 31s 2s/step - loss: 0.9563 - categorical_accuracy: 0.6367 - val_loss: 0.9089 - val_categorical_accuracy: 0.7500 - lr: 1.0000e-05
Epoch 3/40
17/17 [==============================] - 31s 2s/step - loss: 0.9475 - categorical_accuracy: 0.6505 - val_loss: 0.8643 - val_categorical_accuracy: 0.6667 - lr: 1.0000e-05
Epoch 4/40
 8/17 [=============>................] - ETA: 16s - loss: 1.0118 - categorical_accuracy: 0.5956
Epoch 00004: saving model to model_init_Conv_3D2024-08-02053957.092705/model_epoch-00004.h5
17/17 [==============================] - 32s 2s/step - loss: 0.9956 - 

## Exp - 4 Increase Epochs

In [24]:
model_ex20.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=50, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 32
Epoch 1/50
 9/17 [==============>...............] - ETA: 28s - loss: 1.6670 - categorical_accuracy: 0.2361
Epoch 00001: saving model to model_init_Conv_3D2024-08-02053957.092705/model_epoch-00001.h5
17/17 [==============================] - ETA: 0s - loss: 1.5982 - categorical_accuracy: 0.2574Source path =  /home/datasets/Project_data/val ; batch size = 32
Batches:  4 Index: 32
17/17 [==============================] - 70s 4s/step - loss: 1.5982 - categorical_accuracy: 0.2574 - val_loss: 1.6164 - val_categorical_accuracy: 0.2188 - lr: 0.0010
Epoch 2/50
17/17 [==============================] - 44s 3s/step - loss: 1.3484 - categorical_accuracy: 0.4665 - val_loss: 1.6082 - val_categorical_accuracy: 0.1667 - lr: 0.0010
Epoch 3/50
 5/17 [=======>......................] - ETA: 28s - loss: 1.2985 - categorical_accuracy: 0.4522
Epoch 00003: saving model to model_init_Conv_3D2024-08-02053957.092705/model_epoch-00003.h5
17/17 [====